### FPN

In [1]:
import torch.nn as nn
import torch.nn.functional as F
import math
import torch

In [2]:
'''
构建基本卷积块
'''


class BasicConv(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 padding=0,
                 bias=True):
        super(BasicConv, self).__init__()
        self.basic = nn.Sequential(
            nn.Conv2d(in_channels,
                      out_channels,
                      kernel_size,
                      stride=stride,
                      padding=padding,
                      bias=bias), nn.BatchNorm2d(out_channels), nn.ReLU(True))

    def forward(self, x):
        y = self.basic(x)
        return y

In [3]:
'''
构建残差基本块:将输出通道数变为原始通道数的4倍
'''


class Bottleneck(nn.Module):
    expansion = 4  # 输出通道递增率

    def __init__(self, in_channels, planes, stride=1, downsample=None): # in_channels 输入通道  planes 基础通道
        super(Bottleneck, self).__init__()
        self.bottleneck = nn.Sequential(
            BasicConv(in_channels, planes, kernel_size=1, bias=False),
            BasicConv(planes,
                      planes,
                      kernel_size=3,
                      stride=stride,
                      padding=1,
                      bias=False),
            nn.Conv2d(planes,
                      self.expansion * planes,
                      kernel_size=1,
                      bias=False), nn.BatchNorm2d(self.expansion * planes))
        self.relu = nn.ReLU(True)
        self.downsample = downsample

    def forward(self, x):
        identiy = x

        out = self.bottleneck(x)
        if self.downsample:
            identiy = self.downsample(x)
        out += identiy
        y = self.relu(out)
        return y

In [4]:
'''
构建FPN网络
'''


class FPN(nn.Module):
    def __init__(self, layers):  # layers保存每个C模块中bottleneck的数量
        super(FPN, self).__init__()
        self.inplanes = 64
        # 预处理的C1模块
        self.preprocess = nn.Sequential(
            nn.Conv2d(3, 64, 7, 2, padding=3, bias=False), nn.BatchNorm2d(64),
            nn.ReLU(True), nn.MaxPool2d(3, 2, 1))

        # 搭建自下而上的C模块
        self.layer2 = self._make_layer(64, layers[0])
        self.layer3 = self._make_layer(128, layers[1], 2)
        self.layer4 = self._make_layer(256, layers[2], 2)
        self.layer5 = self._make_layer(512, layers[3], 2)

        # 横向连接 保证通道相同
        self.later5 = nn.Conv2d(2048, 256, 1, 1, 0)  # 512*4
        self.later4 = nn.Conv2d(1024, 256, 1, 1, 0)  # 256*4
        self.later3 = nn.Conv2d(512, 256, 1, 1, 0)  # 128*4
        self.later2 = nn.Conv2d(256, 256, 1, 1, 0)  # 64*4

        # 特征融合 3*3
        self.smooth = nn.Conv2d(256, 256, 3, 1, 1)

    def _make_layer(self, planes, numblocks, stride=1):  # planes 输出通道
        downsample = None
        if stride != 1 or self.inplanes != Bottleneck.expansion * planes:  # 输入与输入通道不同
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes,
                          Bottleneck.expansion * planes,
                          1,
                          stride,
                          bias=False),
                nn.BatchNorm2d(Bottleneck.expansion * planes))
        layers = []
        layers.append(Bottleneck(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * Bottleneck.expansion  # 保证 最后通道相同，就不需要downsample模块了
        for i in range(1, numblocks):
            layers.append(Bottleneck(self.inplanes, planes))
        return nn.Sequential(*layers)

    # 上采样，放大特征图，保证特征图大小相同可以相加  让x放大到y的大小，采用插值法
    def _upsample_add(self, x, y):
        _, _, H, W = y.shape
        return F.interpolate(x, size=(H, W), mode='bilinear',align_corners=False) + y  # upsample 被废除了

    # 前向计算
    def forward(self, x):
        # 自下而上
        c1 = self.preprocess(x)
        c2 = self.layer2(c1)
        c3 = self.layer3(c2)
        c4 = self.layer4(c3)
        c5 = self.layer5(c4)

        # 自上而下
        p5 = self.later5(c5)
        p4 = self._upsample_add(p5, self.later4(c4))
        p3 = self._upsample_add(p4, self.later3(c3))
        p2 = self._upsample_add(p3, self.later2(c2))

        # 横向连接，卷积融合
        p4 = self.smooth(p4)
        p3 = self.smooth(p3)
        p2 = self.smooth(p2)
        return p2, p3, p4, p5

In [5]:
fpn=FPN([3,4,6,3])

In [6]:
fpn

FPN(
  (preprocess): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Bottleneck(
      (bottleneck): Sequential(
        (0): BasicConv(
          (basic): Sequential(
            (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace)
          )
        )
        (1): BasicConv(
          (basic): Sequential(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace)
          )
        )
        (2): Conv2d(

In [7]:
'''
查看FPN的第一层
'''

fpn.preprocess

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [8]:
fpn.layer2[0].bottleneck[0].basic[0]

Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)

In [9]:
x = torch.randn(1, 3, 224, 224)
y = fpn(x)  # 返回的y是一个元组
# 输出4个卷积特征图
y[0].shape

torch.Size([1, 256, 56, 56])